In [1]:
!pip install transformers
!pip install langchain
!pip install datasets
!pip install torch torchvision torchaudio
from datasets import load_dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
data_files = {
    "train": "XsumTrain.csv",
    "test": "XsumTest.csv",
    "validation": "XsumValidation.csv"
}

dataset = load_dataset('csv', data_files=data_files)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")

def preprocess_function(examples):
    inputs = examples['document']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


In [ ]:
from transformers import Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig

# Load the model
model = transformers.AutoModelForCausalLM.from_pretrained(
  'tiiuae/falcon-7b-instruct',
  trust_remote_code=True
)

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Reduced batch size
    per_device_eval_batch_size=1,   # Reduced batch size
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=10_000,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=8,  # Accumulate gradients over 8 steps
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)

# Train the model
trainer.train()


In [ ]:
from transformers import pipeline
from langchain import HuggingFacePipeline, PromptTemplate,  LLMChain


# Set up the pipeline for text generation
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=100, do_sample=True, use_cache=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)

# Wrap the pipeline in HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

# Define the prompt template
template = """
Write a concise summary of the following text delimited by triple backquotes.
```{text}```
SUMMARY:
"""

prompt = PromptTemplate(template=template, input_variables=["text"])

# Create LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)


In [ ]:
from transformers import pipeline
from langchain import HuggingFacePipeline, PromptTemplate,  LLMChain


# Set up the pipeline for text generation
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=100, do_sample=True, use_cache=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)

# Wrap the pipeline in HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

# Define the prompt template
template = """
Write a concise summary of the following text delimited by triple backquotes.
```{text}```
SUMMARY:
"""

prompt = PromptTemplate(template=template, input_variables=["text"])

# Create LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.1/774.1 kB 12.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 13.7 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.1 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/databricks/python/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/local_disk0/.ephemeral_nfs/envs/pythonEnv-f5f3b5eb-65bc-4e4e-9543-db2f5e422fea/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/local_disk0/.ephemeral_nfs/envs/pythonEnv-f5f3b5eb-65bc-4e4e-9543-db2f5e422fea/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
df = pd.DataFrame(dataset['test'])[['document', 'summary','id']]

In [ ]:
df.head()

,document,summary,id
0,Prison Link Cymru had 1099 referrals in 201516...,"There is a ""chronic"" need for more housing for...",38264402
1,Officers searched properties in the Waterfront...,"A man has appeared in court after firearms, am...",34227252
2,Jordan Hill Brittany Covington and Tesfaye Coo...,Four people accused of kidnapping and torturin...,38537698
3,The 48yearold former Arsenal goalkeeper played...,West Brom have appointed Nicky Hammond as tech...,36175342
4,Restoring the function of the organ which hel...,The pancreas can be triggered to regenerate it...,39070183


In [ ]:
df.drop(columns=['id'], inplace=True)
df.head()

,document,summary
0,Prison Link Cymru had 1099 referrals in 201516...,"There is a ""chronic"" need for more housing for..."
1,Officers searched properties in the Waterfront...,"A man has appeared in court after firearms, am..."
2,Jordan Hill Brittany Covington and Tesfaye Coo...,Four people accused of kidnapping and torturin...
3,The 48yearold former Arsenal goalkeeper played...,West Brom have appointed Nicky Hammond as tech...
4,Restoring the function of the organ which hel...,The pancreas can be triggered to regenerate it...


In [ ]:
# Create an empty column 'model_generated' in test_df to store the generated summaries
df['model_generated'] = ""

# Define a function to generate summaries and populate the 'model_generated' column
def generate_and_store_summary(row):
    article_text = row['document']
    summary = llm_chain.run(article_text)
    return summary

In [ ]:
df['model_generated'] = df.apply(generate_and_store_summary, axis=1)

In [ ]:
print(df[['document', 'model_generated']].head(25))

                                             document                                    model_generated
0   Prison Link Cymru had 1099 referrals in 201516...  \n              Write a concise summary of the...
1   Officers searched properties in the Waterfront...  \n              Write a concise summary of the...
2   Jordan Hill Brittany Covington and Tesfaye Coo...  \n              Write a concise summary of the...
3   The 48yearold former Arsenal goalkeeper played...  \n              Write a concise summary of the...
4   Restoring the function of the organ  which hel...  \n              Write a concise summary of the...
5   But there certainly should be.\nThese are two ...  \n              Write a concise summary of the...
6   Media playback is not supported on this device...  \n              Write a concise summary of the...
7   Its no joke. But Kareem Badr says people did l...  \n              Write a concise summary of the...
8   Relieved that the giant telecoms company would...  

In [ ]:
# Apply the logic to all rows of the 'model_generated' column
df['model_generated'] = df['model_generated'].apply(lambda text: text.split('SUMMARY:')[1].strip() if 'SUMMARY:' in text else text)

In [ ]:
df['model_generated'][7]

'This article is about The art of public speaking. It discusses the 5 most important tips for aspiring presenters and their first tip is to practice your public speaking skills regularly and take on the challenge of improving you skills.\n1. Practice your speech as much as possible.\n2. Get feedback on your speeches.\n3. Understand your audiences needs.\n4. Engage your audience.\n5. Use media to your advantages.\n\nThe first tip for public speaking is to practice your presentation'

In [ ]:
df['document'][7]

'Its no joke. But Kareem Badr says people did laugh in 2009 when he and two friends paid 20000 Â13000 for the Hideout in Austin when it wasnt making money and the previous owner decided not to renew the lease.\nWe took over a sinking ship and each brought a bucket to bail it out says Mr Badr.\nNone of us had any experience of running a business. But we loved what we were doing enough that it carried us through.\nThree years ago he was able to quit his day job and draw a salary from the club.\nMr Badr says its still not as much as he used to make as a programmer about 80000 a year but he now employs around 25 part time and contract workers.\nAnd he recently expanded the premises taking over the adjoining coffee house which sells alcohol and leasing more theatre space.\nMr Badr says I think my background in computer science helped because I can take a big problem break it up into small chunks and figure out how to make it better and more efficient.\nThats basically what we did for every 

In [5]:
!pip install rouge
!pip install nltk
!pip install bert_score

from rouge import Rouge

# Initialize the ROUGE evaluator
rouge = Rouge()

sampled_df = df

generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['summary'].tolist()

# Calculate ROUGE scores
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

from nltk.translate.bleu_score import corpus_bleu

# Extract the generated summaries and reference summaries
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['summary'].tolist()

# Calculate BLEU score
bleu_score = corpus_bleu(reference_summaries, generated_summaries)

from bert_score import score

# Extract the generated summaries and reference summaries
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['summary'].tolist()

# Calculate BERT Score
P, R, F1 = score(generated_summaries, reference_summaries, lang="en", verbose=True)


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 108.91 seconds, 0.23 sentences/sec


In [6]:
print("ROUGE Scores:", rouge_scores)
print("BLEU Score Summaries:", bleu_score)
print("BERT Precision:", P.mean().item())
print("BERT Recall:", R.mean().item())
print("BERT F1 Score:", F1.mean().item())

ROUGE Scores: {'rouge-1': {'r': 0.27325732278071757, 'p': 0.10613480343655533, 'f': 0.15023619119652162}, 'rouge-2': {'r': 0.027487335987026393, 'p': 0.00971654869268867, 'f': 0.013988781536672719}, 'rouge-l': {'r': 0.22933039710024503, 'p': 0.08878675389736798, 'f': 0.12569667271746987}}
BLEU Score Summaries: 9.212320578498223e-232
BERT Precision: 0.7847375273704529
BERT Recall: 0.8650155663490295
BERT F1 Score: 0.8219874501228333
